# SemEval 2026 Task 8 - Task A: Retrieval (All Domains)

This notebook implements **Task A: Passage Retrieval** for MTRAGEval.

**Goal:** Retrieve relevant documents for **ALL 4 DOMAINS** (`govt`, `clapnq`, `fiqa`, `cloud`).

**Process:**
1. Iterate through each domain.
2. Build/Load dedicated Qdrant collection.
3. Filter conversations for that domain.
4. Retrieve and Rerank.
5. Save combined results.

**Output:** Single JSONL file with all results.


## 1. Setup

In [1]:
import os
import sys
import json
import zipfile
from tqdm import tqdm
from pathlib import Path

if os.path.exists("src"):
    PROJECT_ROOT = os.getcwd()
else:
    PROJECT_ROOT = os.path.abspath("..")

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.ingestion import load_and_chunk_data, build_vector_store
from src.retrieval import get_retriever
from qdrant_client import QdrantClient

/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- CONFIGURATION ---
TEAM_NAME = "Gbgers"
DOMAINS = ["govt", "clapnq", "fiqa", "cloud"]  # All domains
TOP_K_RETRIEVE = 20
TOP_K_RERANK = 5

# TEST MODE: Set to True for quick verification
TEST_MODE = True
TEST_SUBSET_SIZE = 1000   # Number of chunks to index per domain (if building new)
TEST_QUERY_LIMIT = 10     # Number of queries to process per domain

CORPUS_BASE_DIR = os.path.join(PROJECT_ROOT, "dataset/corpora/passage_level")
CONVERSATIONS_FILE = os.path.join(PROJECT_ROOT, "dataset/human/conversations/conversations.json")
QDRANT_PATH = os.path.join(PROJECT_ROOT, "qdrant_db")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data/submissions")
OUTPUT_FILE = os.path.join(OUTPUT_DIR, f"submission_TaskA_{TEAM_NAME}.jsonl")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Processing domains: {DOMAINS}")
if TEST_MODE:
    print(f"⚠️ TEST MODE ACTIVE: Indexing only {TEST_SUBSET_SIZE} chunks, processing {TEST_QUERY_LIMIT} queries.")

Processing domains: ['govt', 'clapnq', 'fiqa', 'cloud']
⚠️ TEST MODE ACTIVE: Indexing only 1000 chunks, processing 10 queries.


## 2. Helper Functions

In [3]:
def extract_last_query(turns):
    """Extract last user question from turns."""
    for turn in reversed(turns):
        if turn.get("role") == "user":
            return turn.get("content", "")
    return ""

def get_corpus_file(domain):
    """Get or extract corpus file path."""
    jsonl_path = os.path.join(CORPUS_BASE_DIR, f"{domain}.jsonl")
    zip_path = os.path.join(CORPUS_BASE_DIR, f"{domain}.jsonl.zip")
    
    if not os.path.exists(jsonl_path):
        if os.path.exists(zip_path):
            print(f"📦 Extracting {domain}.jsonl...")
            with zipfile.ZipFile(zip_path, 'r') as zf:
                zf.extractall(CORPUS_BASE_DIR)
        else:
            return None
    return jsonl_path

def ensure_vector_store(domain):
    """Ensure Qdrant collection exists for domain."""
    collection_name = f"mtrag_{domain}"
    corpus_path = get_corpus_file(domain)
    
    if not corpus_path:
        print(f"⚠️ Corpus not found for {domain}")
        return None
    
    need_build = True
    if os.path.exists(QDRANT_PATH):
        client = None
        try:
            client = QdrantClient(path=QDRANT_PATH)
            if client.collection_exists(collection_name):
                info = client.get_collection(collection_name)
                print(f"✅ Collection '{collection_name}' exists ({info.points_count} vectors)")
                need_build = False
        except Exception as e:
            print(f"⚠️ Warning checking collection: {e}")
        finally:
            if client:
                client.close()  # Ensure lock is released
    
    if need_build:
        print(f"🔄 Building collection '{collection_name}' for {domain}...")
        docs = load_and_chunk_data(corpus_path)
        
        if TEST_MODE and len(docs) > TEST_SUBSET_SIZE:
            print(f"✂️ TEST MODE: Slicing to first {TEST_SUBSET_SIZE} chunks (from {len(docs)}) ")
            docs = docs[:TEST_SUBSET_SIZE]
            
        build_vector_store(docs, persist_dir=QDRANT_PATH, collection_name=collection_name)
        print("✅ Built and saved")
    
    return collection_name

## 3. Main Processing Loop

Iterate over domains, build stores, and run retrieval.

In [4]:
# Load ALL conversations once
print("📂 Loading conversations...")
with open(CONVERSATIONS_FILE, 'r') as f:
    all_conversations = json.load(f)
print(f"Total conversations: {len(all_conversations)}")

📂 Loading conversations...
Total conversations: 110


In [ ]:
all_results = []

for domain in DOMAINS:
    print(f"\n{'='*40}\n🌍 PROCESSING DOMAIN: {domain.upper()}\n{'='*40}")
    
    # 1. Setup Vector Store
    try:
        collection_name = ensure_vector_store(domain)
        if not collection_name:
            continue
    except Exception as e:
        print(f"❌ Critical error setting up vector store for {domain}: {e}")
        continue
        
    # 2. Filter Conversations (FIXED: Substring matching)
    domain_convs = [
        c for c in all_conversations 
        if domain.lower() in c.get("domain", "").lower()
    ]
    print(f"Found {len(domain_convs)} conversations for {domain}")
    
    if not domain_convs:
        continue
        
    if TEST_MODE:
        print(f"✂️ TEST MODE: Processing only first {TEST_QUERY_LIMIT} conversations")
        domain_convs = domain_convs[:TEST_QUERY_LIMIT]
    
    # 3. Init Retriever
    retriever = get_retriever(
        qdrant_path=QDRANT_PATH,
        collection_name=collection_name,
        top_k_retrieve=TOP_K_RETRIEVE,
        top_k_rerank=TOP_K_RERANK
    )
    
    # 4. Run Retrieval
    print(f"🚀 Running retrieval for {domain}...")
    for conv in tqdm(domain_convs):
        query = extract_last_query(conv.get("turns", []))
        if not query: 
            continue
            
        try:
            docs = retriever.invoke(query)
        except Exception as e:
            print(f"Error: {e}")
            docs = []
            
        # Format
        contexts = []
        for i, doc in enumerate(docs):
            meta = doc.metadata
            contexts.append({
                "document_id": str(meta.get("doc_id") or meta.get("parent_id") or f"{domain}_{i}"),
                "score": float(meta.get("relevance_score") or 0.0),
                "text": meta.get("parent_text") or doc.page_content
            })
            
        all_results.append({
            "conversation_id": conv.get("conversation_id"),
            "task_id": f"{conv.get('conversation_id')}::1",
            "Collection": f"mt-rag-{domain}",
            "input": [{"speaker": t["role"], "text": t["content"]} for t in conv["turns"]],
            "contexts": contexts
        })


🌍 PROCESSING DOMAIN: GOVT
✅ Collection 'mtrag_govt' exists (1280 vectors)
Found 28 conversations for govt
✂️ TEST MODE: Processing only first 10 conversations


/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/src/retrieval.py:49: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


## 4. Save Results

In [ ]:
print(f"\n💾 Saving {len(all_results)} total results to {OUTPUT_FILE}...")
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for item in all_results:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')
print("✅ Done!")